## Description

- Agent: particles
- Action: Choose w, c1, c2 through GNN
- Enviroment: Function & The way PSO change state between step (episode)
- State: (X, V, P, G)
- Reward: - f(X)
- Policy: GNN

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import time
import pickle

import torch
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from base_function import *
from utils import *

In [17]:
torch.cuda.empty_cache()

In [18]:
class PSOEnvironment:
    def __init__(self, num_particle, device, lower_bound=-5, upper_bound=5, max_steps=10):
        self.device = torch.device(device)
        self.num_particle = num_particle
        self.lower_bound = torch.tensor(lower_bound, device=self.device)
        self.upper_bound = torch.tensor(upper_bound, device=self.device)
        self.max_steps = max_steps
        self.batch = None
        self.current_step = 0
    
    def set_batch(self, batch):
        self.batch = batch
        self.batch_size = len(batch)
        self.dims = [f['dim'] for f in batch]
        self.func_types = [f['func_type'] for f in batch]
        self.params_list = [f['params'] for f in batch]
    
    def _initialize_population(self):
        self.X = [torch.empty(self.num_particle, d, device=self.device).uniform_(self.lower_bound, self.upper_bound) for d in self.dims]
        self.V = [torch.zeros_like(x) for x in self.X]
        self.P = [x.clone() for x in self.X]
        self.P_best = [torch.full((self.num_particle, 1), float('inf'), device=self.device) for _ in self.dims]
        self.G = [x[0].clone().detach() for x in self.X]
        self.global_best = [float('inf') for _ in self.dims]
        self.fitnesses = [torch.full((self.num_particle, 1), float('inf'), device=self.device) for _ in self.dims]

    def reset(self):
        if not self.batch:
            raise ValueError("Batch is not set. Call set_batch(batch) before reset().")
        self._initialize_population()
        self.current_step = 0

    def evaluate(self, x, func_type, params):
        return Function.get_function(func_type, x, params, self.device).evaluate_function()

    def function(self, x, func_type, params):
        function_instance = Function.get_function(func_type, x, params, self.device)
        return function_instance.evaluate_function().to(self.device)
    
    def step(self, action):
        
        for i in range(self.batch_size):
            W = self.action[i, :, 0].unsqueeze(1).to(self.device)
            C1 = self.action[i, :, 1].unsqueeze(1).to(self.device)
            C2 = self.action[i, :, 2].unsqueeze(1).to(self.device)
            
            X = self.X[i].to(self.device)
            V = self.V[i].to(self.device)
            P = self.P[i].to(self.device)
            
            V_new = (W * V + C1 * (P - X) + C2 * (self.G[i] - X))

            X_new = torch.clamp(X + V_new, self.lower_bound, self.upper_bound)

            self.fitnesses[i] = self.function(X_new, self.func_types[i], self.params_list[i]).to(self.device)
            fitnesses_no_grad = self.fitnesses[i].detach()
            self.V[i] = V_new.detach()
            self.X[i] = X_new.detach()

            improve = fitnesses_no_grad < self.P_best[i]
            self.P_best[i] = torch.where(improve, fitnesses_no_grad, self.P_best[i])
            self.P[i] = torch.where(improve.reshape(-1, 1), self.X[i].detach(), self.P[i])

            min_fitness, min_id = torch.min(fitnesses_no_grad, dim=0)
            if self.G[i] is None or self.global_best[i] > min_fitness:
                self.global_best[i] = min_fitness
                self.G[i] = self.X[i][min_id].detach()
        
        reward = -torch.cat(self.fitnesses, dim=0).to(self.device)
        reward = reward.view(self.batch_size, self.num_particle)
        
        return reward
    

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
from torch_geometric.nn import GCNConv

class GNNPolicy(nn.Module):
    def __init__(self, input_dims, hidden_dims):
        super().__init__()
        self.conv1 = GCNConv(input_dims, hidden_dims)
        self.conv2 = GCNConv(hidden_dims, hidden_dims)
        # Lớp fc trả về 6 giá trị: 3 cho mu, 3 cho sigma (dạng 'thô' hoặc logits)
        self.fc = nn.Linear(hidden_dims, 6)

    def forward(self, x, edge_index):
        """
        x: Tensor shape (B, n, d)
            - B: batch_size
            - n: số node / điểm trong mỗi graph
            - d: số chiều đặc trưng cho mỗi node
        edge_index: Tensor shape (2, E) mô tả các cạnh (trong torch_geometric)
        """
        B, n, d = x.size()
        x = x.view(B * n, d)
        device = x.device

        # Ghép edge_index cho B graph riêng thành 1 edge_index chung
        edge_index_list = []
        for i in range(B):
            offset = i * n
            edge_index_list.append(edge_index + offset)
        edge_index_batch = torch.cat(edge_index_list, dim=1).to(device)

        # GCN forward
        x = F.relu(self.conv1(x, edge_index_batch))
        x = F.relu(self.conv2(x, edge_index_batch))
        x = self.fc(x)  # [B*n, 6]

        # Tách thành mu (3) và sigma (3)
        mu, raw_sigma = x.split(3, dim=-1)  # mỗi cái [B*n, 3]

        # Đưa về [B, n, 3]
        mu = mu.view(B, n, 3)
        raw_sigma = raw_sigma.view(B, n, 3)

        # Xử lý mu, sigma:
        mu = torch.sigmoid(mu)  
        sigma = torch.sigmoid(raw_sigma) * 0.5 + 1e-5  # đảm bảo sigma > 0

        # Xử lý NaN (nếu có)
        mu = torch.where(torch.isnan(mu), torch.zeros_like(mu), mu)
        sigma = torch.where(torch.isnan(sigma), torch.zeros_like(sigma), sigma)

        # Tạo ma trận covariance dạng chéo: (B, n, 3, 3)
        cov = torch.diag_embed(sigma**2)

        # Tạo phân phối MultivariateNormal 3D (mỗi node là 1 phân phối 3D)
        dist_3d = MultivariateNormal(loc=mu, covariance_matrix=cov)
        return dist_3d

    def sample_action(self, x, edge_index):
        """
        - Trả về action shape (B, n, 3)
        - log_prob shape (B, n)
        """
        dist_3d = self.forward(x, edge_index)
        action = dist_3d.rsample()         # (B, n, 3)
        log_prob = dist_3d.log_prob(action)  # (B, n)
        return action, log_prob


In [20]:
from torch.nn.utils import clip_grad_norm_
def train(dataloader, env, policy_net, optimizer, padding_dim, num_epochs, batch_size, num_steps, gamma, device):
    max_grad_norm = 1.0
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        epoch_loss = 0
        num_batches = len(dataloader) // batch_size + (1 if len(dataloader) % batch_size > 0 else 0)
        function_loader = create_batches(dataloader, batch_size)
        epoch_losses = []
        
        for batch_id, batch in enumerate(function_loader):
            env.set_batch(batch)
            env.reset()
            log_probs = []
            rewards = []
            lambda_reg = 1e-4
            for step in range(num_steps):
                X_padded = torch.stack(padding_batch(env.X, batch, padding_dim), dim=0)
                edge_index = torch.combinations(torch.arange(env.num_particle, device=device), r=2).t().contiguous().to(device)
                action, log_prob = policy_net.sample_action(X_padded, edge_index)
                log_probs.append(log_prob)
                env.action = torch.sigmoid(action)
                reward = env.step(env.action).to(device)
                # print(f"Step {step+1} - Raw Reward: {reward}")
                reward = (reward - reward.mean()) / (reward.std() + 1e-8)
                rewards.append(reward)

            discounted_rewards = []
            cumulative_reward = torch.zeros_like(rewards[0])
            for reward in reversed(rewards):
                cumulative_reward = reward + gamma * cumulative_reward
                discounted_rewards.insert(0, cumulative_reward)
            discounted_rewards = torch.stack(discounted_rewards)
            # print(f"Discounted Rewards: {discounted_rewards}")
            # Normalize discounted rewards and make them non-negative
            discounted_rewards = (discounted_rewards - discounted_rewards.mean(dim=0)) / (discounted_rewards.std(dim=0) + 1e-8)
            discounted_rewards = discounted_rewards - discounted_rewards.min(dim=0, keepdim=True)[0]

            loss_components = []
            for lp, dr in zip(log_probs, discounted_rewards):
                component_loss = (-lp * dr).mean()
                loss_components.append(component_loss)
                # print(f"Log Prob: {lp}, Discounted Reward: {dr}, Component Loss: {component_loss}")  # Debug components
            loss = sum(loss_components)/len(loss_components)
            # loss += lambda_reg * sum(p.pow(2).sum() for p in policy_net.parameters())
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()

            print(f"Batch {batch_id + 1}/{num_batches} - Loss: {loss.item():.4f}")
        
        epoch_loss = epoch_loss / batch_size
        epoch_end_time = time.time()
        epoch_losses.append(epoch_loss)
        
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f}, Time: {epoch_end_time - epoch_start_time:.2f}s")
    avg_loss = sum(epoch_losses) / num_epochs
    return avg_loss


In [ ]:
import torch
import pickle
import torch.optim as optim

def main():
    splited_path = r'A:\Code\pso_rl\splited_data_no.pkl'
    with open(splited_path, 'rb') as f:
        dataset = pickle.load(f)

    train_set = dataset['train']
    test_set = dataset['test']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = 64
    num_steps = 25
    padding_dim = 100
    num_particle = 10
    
    # Khởi tạo môi trường RL
    env = PSOEnvironment(num_particle=num_particle, device=device, lower_bound=-5, upper_bound=5, max_steps=num_steps)
    policy_net = GNNPolicy(input_dims=padding_dim, hidden_dims=32).to(device)
    optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)
    num_epochs = 4
    gamma = 0.99
    
    losses = train(
        train_set, 
        env, 
        policy_net, 
        optimizer, 
        padding_dim, 
        num_epochs, 
        batch_size, 
        num_steps, 
        gamma, 
        device
    )

    print("Training completed. Losses:", losses)

    model_save_path = r'A:\Code\pso_rl\data\pso_rl_model.pth'
    torch.save(policy_net.state_dict(), model_save_path)
    print(f"Model saved at {model_save_path}")

if __name__ == "__main__":
    main()


In [ ]:
import torch
import random
import numpy as np
import collections
from statistics import mean, stdev
import pickle

def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def load_model(model_path, policy_net):
    try:
        state_dict = torch.load(model_path)
        policy_net.load_state_dict(state_dict)
        print(f"Model loaded successfully from {model_path}")
    except Exception as e:
        print(f"Error loading the model: {e}")
        raise e
    return policy_net

def test(test_set, env, policy_net, padding_dim, num_steps, gamma, device):
    import collections
    from statistics import mean, stdev

    func_rewards = { 'ackley': [], 'levy': [], 'largeman': [], 'griewank': [], 'styblinski_tang':[]}
    step_convergence = { 'ackley': [], 'levy': [], 'largeman': [], 'griewank': [], 'styblinski_tang': []}

    function_loader = create_batches(test_set, batch_size=32)

    for batch_id, batch in enumerate(function_loader):
        env.set_batch(batch)
        env.reset()
        log_probs = []
        cumulative_rewards = collections.defaultdict(list)
        final_step_rewards = collections.defaultdict(list)
        no_improve_steps = collections.defaultdict(int)
        converged_func_types = set()  # Initialize outside the loop
        prev_global_best = None  # Initialize outside the loop
        max_no_improve = 6  # Termination condition if no improvement for 6 steps

        for step in range(num_steps):
            X_padded = torch.stack(padding_batch(env.X, batch, padding_dim), dim=0)
            edge_index = torch.combinations(torch.arange(env.num_particle, device=device), r=2).t().contiguous()
            action, log_prob = policy_net.sample_action(X_padded, edge_index)
            log_probs.append(log_prob)
            env.action = torch.sigmoid(action)
            reward = env.step(env.action).to(device)

            # Initialize prev_global_best after the first step
            if step == 0:
                prev_global_best = [env.global_best[i].item() for i in range(len(env.func_types))]
                continue  # Skip convergence check for step 0

            # Check for convergence for each func_type
            for i, func_type in enumerate(env.func_types):
                if func_type in converged_func_types:  # Skip if already converged
                    continue

                current_best = env.global_best[i].item()
                if current_best >= prev_global_best[i]:  # No improvement (we are minimizing)
                    no_improve_steps[func_type] += 1
                else:
                    no_improve_steps[func_type] = 0

                if no_improve_steps[func_type] >= max_no_improve:
                    # print(f"Convergence detected for {func_type} at step {step}. Early stopping for this function type.")
                    if func_type not in step_convergence or len(step_convergence[func_type]) == 0:
                        step_convergence[func_type].append(step)
                    converged_func_types.add(func_type)  # Mark as converged

            prev_global_best = [env.global_best[i].item() for i in range(len(env.func_types))]

            # Optional: Early stopping if all function types have converged
            if len(converged_func_types) == len(env.func_types):
                break

            # Track rewards for each function type at each step
            for i, func_type in enumerate(env.func_types):
                reward_values = reward[i].tolist()  # Convert tensor to list of scalars
                cumulative_rewards[func_type].extend(reward_values)
                if step == num_steps - 1 or no_improve_steps[func_type] >= max_no_improve:
                    final_step_rewards[func_type].extend(reward_values)

        # Store the final global_best as the best reward for the batch
        for i, func_type in enumerate(env.func_types):
            func_rewards[func_type].append(env.global_best[i].item())

    # Log final statistics
    print("\nFinal Statistics:")
    for func_type in func_rewards:
        avg_reward = mean(func_rewards[func_type])
        std_reward = stdev(func_rewards[func_type]) if len(func_rewards[func_type]) > 1 else 0.0
        avg_convergence = mean(step_convergence[func_type]) if step_convergence[func_type] else -1
        print(f"Function {func_type}: Average Reward: {avg_reward:.4f} ± {std_reward:.4f}, Average Convergence Step: {avg_convergence:.2f}")

    # Calculate best and worst results from final timestep rewards
    best_rewards = {ftype: max(values) for ftype, values in final_step_rewards.items()}
    worst_rewards = {ftype: min(values) for ftype, values in final_step_rewards.items()}

    print("\nBest Results (Final Timestep):")
    for func_type, reward in best_rewards.items():
        print(f"Function {func_type}: Best Reward: {reward:.4f}")

    print("\nWorst Results (Final Timestep):")
    for func_type, reward in worst_rewards.items():
        print(f"Function {func_type}: Worst Reward: {reward:.4f}")

    convergence_results = {ftype: -max(values) for ftype, values in func_rewards.items()}
    print("\nConvergence Results:")
    for func_type, convergence in convergence_results.items():
        print(f"Function {func_type}: {convergence:.4f}")

    print("\nDimensional Analysis (from env.dims):")
    print(f"Dimensions: {env.dims}")

    return {func_type: (mean(func_rewards[func_type]), stdev(func_rewards[func_type]) if len(func_rewards[func_type]) > 1 else 0.0) for func_type in func_rewards}

# Main function
if __name__ == "__main__":
    set_seed(625)
    gamma = 0.99
    num_steps = 50
    padding_dim = 100
    model_path = r"A:\Code\pso_rl\data\pso_rl_model.pth"
    splited_path = r'A:\Code\pso_rl\splited_data_no.pkl'

    # Load dataset
    with open(splited_path, 'rb') as f:
        dataset = pickle.load(f)
    train_set = dataset['train']
    test_set = dataset['test']

    # Initialize environment and model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_particle = 10
    env = PSOEnvironment(num_particle=num_particle, device=device, lower_bound=-5, upper_bound=5, max_steps=num_steps)
    policy_net = GNNPolicy(input_dims=padding_dim, hidden_dims=32).to(device)

    # Load the trained model
    policy_net = load_model(model_path, policy_net)

    # Test the model
    best_rewards = test(
        test_set=test_set, 
        env=env, 
        policy_net=policy_net, 
        padding_dim=padding_dim, 
        num_steps=num_steps, 
        gamma=gamma, 
        device=device
    )

    print(best_rewards)


In [ ]:
print(torch.__version__)